In [1]:
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.eventScheduler import *
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-04-05 22:08:04 trade.helpers.Logging INFO: Logging Root Directory: c:\Users\Zino\python-playground\QuantTools\logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
import json
with open('../input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
ttrades_ = pd.read_csv('../input/profitable_trades.csv').iloc[:, 1:]

w_map = {x: w  * 0.75 for x, w in weights.items()}
ttrades_ = ttrades_[ttrades_['EntryTime'] != ttrades_['ExitTime']]
ttrades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT
10,38.0,535.0,580.0,226.851208,217.009995,-373.966096,-0.043382,2023-08-17,2023-10-20,64 days,TSLA


In [3]:
new_trades = ttrades_.copy()[:10]

In [ ]:
trades_ = ttrades_.copy()[:5]
msft_trade = pd.DataFrame([{"Size": -10.0, "EntryBar": 504, "ExitBar": 529.0, "EntryTime": "2023-07-05", "ExitTime": "2023-08-10", "Ticker": "MSFT"}])
trades_ = pd.concat([trades_, msft_trade], ignore_index=True)
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,10.0,504.0,529.0,NaN,NaN,NaN,NaN,2023-08-09,2023-10-25,NaN,MSFT


In [8]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_)

In [9]:
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.1},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.1}],

 'name': 'vertical_spread'}

evb_backtest.portfolio.order_settings

The following symbols: ['NVDA', 'QCOM', 'MU', 'GOOG', 'AMD', 'BAC', 'BA', 'SBUX', 'DIS', 'HD', 'WMT', 'PFE', 'JNJ', 'INTC'] are not being processed but present in weight_map


{'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': 0.9,
   'dte': 365,
   'moneyness_width': 0.1},
  {'direction': 'short',
   'rel_strike': 0.8,
   'dte': 365,
   'moneyness_width': 0.1}],
 'name': 'vertical_spread'}

In [10]:
evb_backtest.events.events

,type,symbol,signal_type,signal_id,max_contract_price,order_settings
datetime,,,,,,
2023-07-05,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-05,SIGNAL,AAPL,LONG,AAPL20230705LONG,NaN,NaN
2023-07-05,SIGNAL,MSFT,LONG,MSFT20230705LONG,NaN,NaN
2023-07-05,SIGNAL,AVGO,LONG,AVGO20230705LONG,NaN,NaN
2023-07-05,SIGNAL,AMZN,LONG,AMZN20230705LONG,NaN,NaN
2023-07-05,SIGNAL,TSLA,LONG,TSLA20230705LONG,NaN,NaN
2023-07-06,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-07,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-10,MARKET,NaN,NaN,NaN,NaN,NaN


In [11]:
evb_backtest.portfolio.roll_map

{'TSLA': 30, 'AAPL': 30, 'MSFT': 30, 'AVGO': 30, 'AMZN': 30}

In [13]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest  

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Saving to cache from db
Buy Details
Position: {'long': ['TSLA20240621C283.33'], 'short': ['TSLA20240621C400'], 'trade_id': '&L:TSLA20240621C283.33&S:TSLA20240621C400', 'close': 34.225}, Date: 2023-07-05, Signal: SignalEvent type:LONG, symbol=TSLA, date:2023-07-05 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:TSLA20230705LONG
Max Contract Price: 41.06437111050783, Cash at Hand: 73.9158679989141
Cash at Hand 73.9158679989141 Close 34.225
Processing event: SIGNAL
Buy Details
Position: {'long': ['AAPL20240621C195'], 'short': ['AAPL20240621C270'], 'trade_id': '&L:AAPL20240621C195&S:AAPL20240621C270', 'close': 18.665}, Date: 2023-07-05, Signal: SignalEvent type:LONG, symbol=AAPL, date:2023-07-05 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:AAPL20230705LONG
Max Contract Price: 23.253140269800777, Cash at Hand: 41.8556524856414
Cash at Hand 41.8556524856414 Close 18.665
Processing event: SIGNAL
Buy Detai

In [14]:
pd.DataFrame(evb_backtest.portfolio.unprocessed_signals)

""


In [15]:
evb_backtest.portfolio.trades

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,TSLA,-1758.723642,-0.256608,3426.867253,1.116,7.618505,6852.618505,68.537345,8.734505,2547.505432,1.116,-3.873137,5096.126863,50.950109,4.989137,2,2023-07-05,2023-08-02,28,&L:TSLA20240621C283.33&S:TSLA20240621C400,TSLA20230705LONG
1,AAPL,-987.803067,-0.264080,1870.271838,1.116,6.427675,3739.427675,37.405437,7.543675,1376.370304,1.116,-2.143392,2753.856608,27.527406,3.259392,2,2023-07-05,2023-08-04,30,&L:AAPL20240621C195&S:AAPL20240621C270,AAPL20230705LONG
2,MSFT,-832.940139,-0.265590,3136.192152,0.558,3.134152,3135.634152,31.361922,3.692152,2303.252013,0.558,-1.189987,2303.810013,23.032520,1.747987,1,2023-07-05,2023-08-09,35,&L:MSFT20240621C345&S:MSFT20240621C450,MSFT20230705LONG
3,AMZN,-481.366979,-0.315370,1526.354935,0.558,0.796935,1525.796935,15.263549,1.354935,1044.987955,0.558,-0.954045,1045.545955,10.449880,1.512045,1,2023-07-05,2023-10-25,112,&L:AMZN20240621C132.5&S:AMZN20240621C185,AMZN20230705LONG
4,AVGO,-1588.026161,-0.241772,3284.140781,1.116,7.165562,6567.165562,65.682816,8.281562,2490.127701,1.116,-8.628599,4981.371401,49.802554,9.744599,2,2023-07-05,2023-08-18,44,&L:AVGO20240119C900&S:AVGO20240119C1000,AVGO20230705LONG
5,MSFT,733.379128,0.232139,3159.226281,0.558,3.668281,3158.668281,31.592263,4.226281,3892.605409,0.558,-2.836591,3893.163409,38.926054,3.394591,1,2023-08-09,2023-10-25,77,&L:MSFT20240621C330&S:MSFT20240621C460,MSFT20230809LONG


In [17]:
evb_backtest.portfolio.get_weighted_holdings()

,TSLA,AAPL,MSFT,AVGO,AMZN,cash,commission,total
datetime,,,,,,,,
2023-07-05,8212.874222,4650.628054,6388.181048,8870.323437,3246.163861,68631.829378,0.00000,100000.000000
2023-07-05,8204.139717,4650.628054,6388.181048,8870.323437,3246.163861,68631.829378,0.01116,99991.265495
2023-07-05,8204.139717,4643.084379,6388.181048,8870.323437,3246.163861,68631.829378,0.02232,99983.721819
2023-07-05,8204.139717,4643.084379,6384.488896,8870.323437,3246.163861,68631.829378,0.02790,99980.029667
2023-07-05,8204.139717,4643.084379,6384.488896,8870.323437,3244.808926,68631.829378,0.03348,99978.674733
2023-07-05,8204.139717,4643.084379,6384.488896,8862.041875,3244.808926,68631.829378,0.04464,99970.393170
2023-07-05,8204.139717,4643.084379,6384.488896,8862.041875,3244.808926,68631.829378,0.04464,99970.393170
2023-07-06,7934.139717,4751.084379,6481.988896,8342.041875,3157.308926,68631.829378,0.04464,99298.393170
2023-07-07,7784.139717,4619.084379,6349.488896,8282.041875,3227.308926,68631.829378,0.04464,98893.893170


In [18]:
evb_backtest.portfolio.get_all_positions()

long                short  \
datetime   symbol                                               
2023-07-05 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
           TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
           TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-06 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-07 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-10 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-11 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-12 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-13 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-14 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-07-17 TSLA    [TSLA20240621C283.33]   [TSLA20240621C400]   
           AAPL       [AAPL20240621C195]   [AAPL20240621C270]   
           MSFT       [MSFT20240621C345]   [MSFT20240621C450]   
           AVGO       [AVGO20240119C900]  [AVGO20240119C1000]   
           AMZN     [AMZN20240621C132.5]   [AMZN20240621C185]   
2023-0

In [ ]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         179946728 function calls (179230715 primitive calls) in 771.171 seconds

   Ordered by: cumulative time
   List reduced from 2996 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  771.173  385.587 c:\Users\Zino\python-playground\QuantTools\.venv\lib\site-packages\IPython\core\interactiveshell.py:3517(run_code)
        2    0.000    0.000  771.173  385.587 {built-in method builtins.exec}
        1    0.000    0.000  771.173  771.173 C:\Users\Zino\AppData\Local\Temp\ipykernel_28000\3226836942.py:1(<module>)
        1    0.028    0.028  771.173  771.173 C:\Users\Zino\python-playground\QuantTools\EventDriven\backtest.py:37(run)
       79    0.000    0.000  629.085    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:378(update_signal)
       79    0.003    0.000  629.083    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:267(generate_order)
       45 

In [ ]:
evb_backtest.portfolio.plot_portfolio()

AttributeError: 'OptionSignalPortfolio' object has no attribute '_trades'

In [ ]:
trades_df = evb_backtest.portfolio.get_trades().head(5)
trades_df

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-10806.520059,-21.677784,498.506687,6.417,16.736810,11459.236810,11488.807620,-29.570810,390.441487,6.417,16.571196,8986.571196,9003.142393,-16.571196,23,2023-07-05,2023-08-02,28,&L:TSLA20240621C330&S:TSLA20240621C346.67
1,AAPL,3656.985131,7.479087,488.961456,3.627,-10.628066,6352.871934,6349.497867,3.374066,525.531308,3.627,-2.465999,6835.534001,6833.068001,2.465999,13,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C280
2,MSFT,-10309.332567,-21.247498,485.202187,3.906,-1.075379,6788.924621,6795.661241,-6.736621,382.108862,3.906,-1.569939,5353.430061,5351.860122,1.569939,14,2023-07-05,2023-08-09,35,&L:MSFT20240621C355&S:MSFT20240621C365
3,AMZN,-5391.261254,-10.873383,495.821865,0.837,1.628594,1486.628594,1489.931189,-3.302594,441.909252,0.837,-0.935243,1326.564757,1325.629513,0.935243,3,2023-07-05,2023-10-25,112,&L:AMZN20240621C145&S:AMZN20240621C160


In [ ]:
evb_backtest.portfolio.get_weighted_holdings()s

In [ ]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                  short  \
datetime   symbol                                              
2023-07-05 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-06 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-07 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-10 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-11 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-12 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-13 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-14 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-17 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-18 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-19 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-20 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-21 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
 

: 